<a href="https://colab.research.google.com/github/limseo12/deep-learning-from-scratch_Study/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94%EB%94%A5%EB%9F%AC%EB%8B%9D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Variable = 변수 , 클래스 구현
class Variable:
  def __init__(self, data):
    self.data = data

import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

#새로운 데이터입력
x.data = np.array(2.0)
print(x.data)

In [ ]:
#ndim 인스턴스 변수 사용해보기 = 차원 수
import numpy as np
x = np.array(1)
x.ndim
#결과 = 0
x = np.array([1, 2, 3])
x.ndim
#결과 = 1
x = np.array([[1, 2, 3],
              [4, 5, 6]])
x.ndim
#결과 = 2

In [ ]:
#2.2 Function 클래스 구현
class Function:
  def __call__(self, input):  #파이썬의 특수 메서드.
    x = input.data
    y = x ** 2
    output = Variable(y)
    return output

#2.3 FUnction 클래스 이용 2.2와 연계
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

In [ ]:
#calss Function 수정
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output
  def forward(self, x):
    raise NotImplementedError()

In [ ]:
#Function 클래스를 상속하여 입력값을 제곱하는 클래스를 구현
class Square(Function):
  def forward(self, x):
    return x ** 2

x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

## step 3 함수 연결

In [ ]:
#3.1 Exp 함수 구현
class Exp(Function):
  def forward(self, x):
    return np.exp(x)

#3.2 함수 연결 y = (e^x^2)^2 이라는 계산을 예로 생각
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

# Step 4 수치 미분

#중앙차분, 전진차분 .
#중앙차분이 상대적으로 오차가 적다
#앱실론의 약어 eps. 아주 작은 양의 부동소수점값을 담는 변수의 이름으로 흔희 사용된다.

In [ ]:
#4,1 중앙차분을 이용하여 수치미분을 계산해보자
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)

# 아까 구현한 Square 클래스 대상으로 미분해보자
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

In [ ]:
#4.3 합성 함수의 미분
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)


4.4 수치미분의 문제점\
'자릿수 누락' 때문에 문제가있다.\
수치미분을 이용하면 자릿수 누락이 발생하여 오차가 포함되기 쉽다.

역전파\
수치 미분은 구현하기 쉽고 거의 정확한 값을 얻을 수 있다.\
이에 비해 역전파는 복잡한 알고리즘이여서 구현하면서 버그가 섞여 들어가기 쉽다.\
역전파를 정확하게 구현했는지 확인하기 위해 수치 미분의 결과를 이용하곤 한다.\
이를 기울기 확인 (gradient checking) 이라고한다\
이것은 단순히 수치 미분 결과와 역전파의 결과를 비교하는 것이다.

# Step 5 역전파 (Back propagation) 이론 

In [ ]:
#5.1 연쇄법칙
